<a href="https://colab.research.google.com/github/HowkMii/GDSC-ML_Academy_Project1/blob/main/ML_Academy_Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Abdelhakim Azzouz & Tasnim Mr.Abulhassan**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import 
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print(tf.__version__)

In [ ]:
cd /content/drive/MyDrive/ML_Academy-Project1/

In [ ]:
ls

We have four data sets that are train and test data sets. 

In [ ]:
# read the data with all the defaults for panda
d_client_train = pd.read_csv('client_train.csv')
d_client_test = pd.read_csv('client_test.csv')
d_invoice_train = pd.read_csv('invoice_train.csv')
d_invoice_test = pd.read_csv('invoice_test.csv')
#d_SampleSubmission = pd.read_csv('SampleSubmission.csv')

In [ ]:
#Now we have all the row
d_invoice_train.head(20)

In [ ]:
d_client_train.head(20)

In [ ]:
total = d_invoice_train .isnull().sum().sort_values(ascending=False)
percent_1 = d_invoice_train .isnull().sum()/d_invoice_train .isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head()

In [ ]:
d_invoice_train.columns.values

In [ ]:
d_invoice_train['reading_remarque'].value_counts()

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
d_invoice_train['counter_type']=lb.fit_transform(d_invoice_train['counter_type'])

In [ ]:
d_invoice_train.head(10)

In [ ]:
d_invoice_train['counter_type'].value_counts()

In [ ]:
sns.countplot(x="counter_type", data=d_invoice_train)

In [ ]:
d_invoice_train=d_invoice_train.drop(['invoice_date','tarif_type','counter_number','counter_statue','counter_code','counter_coefficient','old_index','new_index','months_number'],axis=1)

In [ ]:
d_invoice_train.head()

i will like to calculate the mean per cluster.

In [ ]:
d_invoice_train = d_invoice_train.groupby("client_id")
d_invoice_train = d_invoice_train.mean()
d_invoice_train.head(5)

In [ ]:
d_client_train = pd.merge(d_client_train, d_invoice_train, on='client_id', how='right')

In [ ]:
d_client_train.head()

Correlation between features and target

In [ ]:
pd.plotting.scatter_matrix(d_client_train, figsize=(12,12));

In [ ]:
d_client_train['client_catg'] = d_client_train['client_catg'].astype('object')
d_client_train['disrict'] = d_client_train['disrict'].astype('object')

# **Same for data Test**

In [ ]:
d_client_test.head(5)

In [ ]:
d_invoice_test.head()

In [ ]:
d_invoice_test['counter_type']=lb.fit_transform(d_invoice_test['counter_type'])

In [ ]:
d_invoice_test=d_invoice_test.drop(['invoice_date','tarif_type','counter_number','counter_statue','counter_code','counter_coefficient','old_index','new_index','months_number'],axis=1)

In [ ]:
d_invoice_test.head()

In [ ]:
d_invoice_test = d_invoice_test.groupby("client_id")
d_invoice_test = d_invoice_test.mean()
d_invoice_test.head(5)

In [ ]:
d_client_test = pd.merge(d_client_test, d_invoice_test, on='client_id', how='right')

In [ ]:
d_client_test.head()

In [ ]:
d_client_test['client_catg'] = d_client_test['client_catg'].astype('object')
d_client_test['disrict'] = d_client_test['disrict'].astype('object')

In [ ]:
for data in [d_client_train,d_client_test]:
    data['creation_date'] = pd.to_datetime(data['creation_date'])

In [ ]:
d_client_train=d_client_train.drop(['client_id', 'creation_date'],axis=1)
d_client_test=d_client_test.drop(['client_id', 'creation_date'],axis=1)

In [ ]:
d_client_test.head()

In [ ]:
d_client_train=d_client_train.drop(['client_catg', 'reading_remarque'],axis=1)
d_client_test=d_client_test.drop(['client_catg', 'reading_remarque'],axis=1)

In [ ]:
d_client_train.head()

In [ ]:
for i in d_client_test.columns:
    if d_client_train[i].dtype=='object' or d_client_test[i].dtype=='object': 
        lb = preprocessing.LabelEncoder()
        lb.fit(list(d_client_train[i].values) + list(d_client_test[i].values))
        d_client_train[i] = lb.transform(list(d_client_train[i].values))
        d_client_test[i] = lb.transform(list(d_client_test[i].values)) 

In [ ]:
d_label=d_client_train['target']
d_feature= d_client_train.drop('target',axis=1)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [ ]:
params = {'boosting' :'gbdt',
                       'learning_rate':0.05,
                       'importance_type':'split',
                       'num_leaves':60,
          'metric':'auc',
                       'max_depth':1,
                       'subsample':0.5,
                       'max_featuers':'log2', 'verbose':1}
X = d_feature
y = d_label
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True)
print("TRAIN:", X_train.shape, y_train.shape, "TEST:", X_valid.shape, y_valid.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#input_layer = Input(shape=(X.shape[1],))
#dense_layer_1 = Dense(14, activation='relu')(input_layer)
#dense_layer_2 = Dense(5, activation='relu')(dense_layer_1)
#output = Dense(2)(dense_layer_2)
#model = Model(inputs=input_layer, outputs=output)
#model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model = tf.keras.Sequential([
layers.Dense(input_dim =7, units = 128, activation = "relu"),
layers.Dense(units= 64, activation = "relu"),
layers.Dropout(0.2),
layers.Dense(units= 32, activation = "relu"),
layers.Dropout(0.2),
layers.Dense(units=1, activation = "sigmoid")])
model.summary()

In [ ]:
from keras import losses, optimizers, metrics
model.compile(optimizer ='adam', loss ='mean_squared_error', metrics =["Accuracy"])

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_s, y_train_s = sm.fit_resample(X_train, y_train)
history = model.fit(X_train_s, y_train_s, batch_size=3, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
54607/54607 [==============================] - 127s 2ms/step - loss: 0.2124 - Accuracy: 0.6644 - val_loss: 0.3465 - val_Accuracy: 0.2990
Epoch 2/50
54607/54607 [==============================] - 125s 2ms/step - loss: 0.2066 - Accuracy: 0.6764 - val_loss: 0.3159 - val_Accuracy: 0.4527
Epoch 3/50
54607/54607 [==============================] - 125s 2ms/step - loss: 0.2045 - Accuracy: 0.6809 - val_loss: 0.3235 - val_Accuracy: 0.4005
Epoch 4/50
54607/54607 [==============================] - 129s 2ms/step - loss: 0.2031 - Accuracy: 0.6840 - val_loss: 0.3441 - val_Accuracy: 0.3701
Epoch 5/50
54607/54607 [==============================] - 131s 2ms/step - loss: 0.2021 - Accuracy: 0.6861 - val_loss: 0.3249 - val_Accuracy: 0.3872
Epoch 6/50
54607/54607 [==============================] - 128s 2ms/step - loss: 0.2011 - Accuracy: 0.6881 - val_loss: 0.3370 - val_Accuracy: 0.4045
Epoch 7/50
54607/54607 [==============================] - 132s 2ms/step - loss: 0.2003 - Accuracy: 0.6900 - val_

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

847/847 [==============================] - 1s 2ms/step - loss: 0.1350 - Accuracy: 0.8292
Test Score: 0.13501593470573425
Test Accuracy: 0.8291819095611572
